In [4]:
# Environment check 
# Return to HW0 if you run into errors in this cell 
# Do not modify this cell 
import os
# assert os.environ['CONDA_DEFAULT_ENV'] == "cs375"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 11

import numpy as np
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer, DataCollatorWithPadding, TrainingArguments)
from datasets import Dataset, load_dataset, load_metric
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

dataset = load_dataset('csv', data_files=r'train\track_a\eng.csv')
dataset2 = load_dataset('knowledgator/events_classification_biotech')

dev_dataset = load_dataset('csv', data_files=r'dev\track_a\eng_a.csv')

helinski_dataset_raw = load_dataset('csv', data_files=r'xed_fixed.csv')

classes2 = [class_ for class_ in dataset2['train'].features['label 1'].names if class_]
print(classes2)

# class_weights = torch.tensor([1.8, 0.7, 1.2, 0.9, 0.9])

# loss_fn = CrossEntropyLoss(weight=class_weights)

classes = ["Anger", "Fear", "Joy", "Sadness", "Surprise"]
class2id2 = {class_:id for id, class_ in enumerate(classes2)}
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for class_, id in class2id.items()}

c:\Users\thato\miniconda3\envs\semeval1\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for knowledgator/events_classification_biotech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/knowledgator/events_classification_biotech
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


['event organization', 'executive statement', 'regulatory approval', 'hiring', 'foundation', 'closing', 'partnerships & alliances', 'expanding industry', 'new initiatives or programs', 'm&a', 'service & product providing', 'new initiatives & programs', 'subsidiary establishment', 'product launching & presentation', 'product updates', 'executive appointment', 'alliance & partnership', 'ipo exit', 'article publication', 'clinical trial sponsorship', 'company description', 'investment in public company', 'other', 'expanding geography', 'participation in an event', 'support & philanthropy', 'department establishment', 'funding round', 'patent publication']


In [5]:
from transformers import AutoTokenizer
 
model_path = 'microsoft/deberta-v3-small'

tokenizer = AutoTokenizer.from_pretrained(model_path, clean_up_tokenization_spaces=True)

In [6]:
# def preprocess_function(example):
#    text = example['text']
#    labels = [0. for i in range(len(classes))]
#    for label in classes:
#        id = class2id[label]
#        labels[id] = float(example[label])
  
#    example = tokenizer(text, truncation=True)
#    example['labels'] = labels
#    return example

examples = []
y_true = []
dev_examples = []
dev_y_true = []
h_examples = []
h_y_true = []
num_folds = 5

for example in dataset['train']:
  examples.append(example['text'])
  y_true.append([float(example['Anger']), float(example['Fear']), float(example['Joy']), float(example['Sadness']), float(example['Surprise'])])

for example in dev_dataset['train']:
  dev_examples.append(example['text'])
  # dev_y_true.append([float(example['Anger']), float(example['Fear']), float(example['Joy']), float(example['Sadness']), float(example['Surprise'])])

for example in helinski_dataset_raw['train']:
  h_examples.append(example['Sentence'])
  h_y_true.append([float(example['Anger']), float(example['Fear']), float(example['Joy']), float(example['Sadness']), float(example['Surprise'])])

# make training and validation sets
examples_train, examples_test, labels_train, labels_test = train_test_split(examples, y_true, test_size=0.05, random_state=42)

# print(examples_train)
# print(examples_test)


train_dataset = Dataset.from_dict( {"text": examples_train, "label": labels_train} )
test_dataset = Dataset.from_dict( {"text": examples_test, "label": labels_test} )
helinski_dataset = Dataset.from_dict( {"text": h_examples, "label": h_y_true} )

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

new_dataset = Dataset.from_dict( {"text": examples, "label": y_true} )
dev_dataset = Dataset.from_dict( {"text": dev_examples} )

# mskf = MultilabelStratifiedKFold(n_splits=num_folds)
# kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

accumulated_trains = []
accumulated_tests = []

  # make Datasets
# train = new_dataset.select(train_index)
# tokenizd_train = new_dataset.map(tokenize_function, batched=True)
# accumulated_trains.append(tokenizd_train)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_traintest = test_dataset.map(tokenize_function, batched=True)
# test = new_dataset.select(val_index)
tokenized_dev = dev_dataset.map(tokenize_function, batched=True)
tokenized_helinski = helinski_dataset.map(tokenize_function, batched=True)
# accumulated_tests.append(tokenized_test)

# tokenized_test = test_dataset.map(preprocess_function)

Map:   0%|          | 0/2629 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/139 [00:00<?, ? examples/s]

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

Map:   0%|          | 0/12243 [00:00<?, ? examples/s]

In [7]:
# # def preprocess_function(example):
# #    text = example['text']
# #    labels = [0. for i in range(len(classes))]
# #    for label in classes:
# #        id = class2id[label]
# #        labels[id] = float(example[label])
  
# #    example = tokenizer(text, truncation=True)
# #    example['labels'] = labels
# #    return example

# examples = []
# y_true = []
# num_folds = 5

# for example in dataset['train']:
#   examples.append(example['text'])
#   y_true.append([float(example['Anger']), float(example['Fear']), float(example['Joy']), float(example['Sadness']), float(example['Surprise'])])

# # make training and validation sets
# # examples_train, examples_test, labels_train, labels_test = train_test_split(examples, y_true, test_size=0.05, random_state=42)

# # print(examples_train)
# # print(examples_test)


# # train_dataset = Dataset.from_dict( {"text": examples_train, "label": labels_train} )
# # test_dataset = Dataset.from_dict( {"text": examples_test, "label": labels_test} )

# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# new_dataset = Dataset.from_dict( {"text": examples, "label": y_true} )

# mskf = MultilabelStratifiedKFold(n_splits=num_folds)
# # kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# accumulated_trains = []
# accumulated_tests = []

# for fold, (train_index, val_index) in enumerate(mskf.split(examples, y_true)):
#   # make Datasets
#   train = new_dataset.select(train_index)
#   tokenizd_train = train.map(tokenize_function, batched=True)
#   accumulated_trains.append(tokenizd_train)
  
#   test = new_dataset.select(val_index)
#   tokenized_test = test.map(tokenize_function, batched=True)
#   accumulated_tests.append(tokenized_test)

# # tokenized_test = test_dataset.map(preprocess_function)

In [8]:
import evaluate
import numpy as np

from transformers import DataCollatorWithPadding

# print(tokenized_dataset)
# tokenized_dataset = tokenized_dataset["train"].train_test_split(test_size=0.6) 
# print(tokenized_dataset)
# training!
model = AutoModelForSequenceClassification.from_pretrained(model_path, problem_type="multi_label_classification", num_labels=5)

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

# mini_tokenized_train = tokenized_train.select(range(100))
# mini_tokenized_test = tokenized_test.select(range(5))

#  trainer = Trainer(model=model, args=training_args, train_dataset=mini_tokenized_train, eval_dataset=mini_tokenized_test)
# trainer = CustomTrainer(model=model, args=training_args, train_dataset=mini_tokenized_train, eval_dataset=mini_tokenized_test)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))
#references=labels.astype(int).reshape(-1))

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(

   model_path, num_labels=len(classes),
           id2label=id2class, label2id=class2id,
                       problem_type = "multi_label_classification")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# for i in range(num_folds):

TESTING_SET = tokenized_helinski

training_args = TrainingArguments(
   output_dir="my_awesome_model",
   per_device_train_batch_size = 8,
   learning_rate= 2e-5,
   # per_device_train_batch_size=3,
   # per_device_eval_batch_size=3,
   num_train_epochs=3,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
)

trainer = Trainer(

   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_traintest,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   # mini_batch_sizee = b_s
)

trainer.train()


c:\Users\thato\miniconda3\envs\semeval1\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/987 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.45140814781188965, 'eval_accuracy': 0.7755395683453238, 'eval_f1': 0.6231884057971014, 'eval_precision': 0.6417910447761194, 'eval_recall': 0.6056338028169014, 'eval_runtime': 3.6509, 'eval_samples_per_second': 38.073, 'eval_steps_per_second': 4.93, 'epoch': 1.0}
{'loss': 0.4872, 'grad_norm': 3.015688896179199, 'learning_rate': 9.868287740628168e-06, 'epoch': 1.52}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.4059754014015198, 'eval_accuracy': 0.8100719424460432, 'eval_f1': 0.6432432432432432, 'eval_precision': 0.7579617834394905, 'eval_recall': 0.5586854460093896, 'eval_runtime': 2.8121, 'eval_samples_per_second': 49.43, 'eval_steps_per_second': 6.401, 'epoch': 2.0}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.3965293765068054, 'eval_accuracy': 0.8244604316546763, 'eval_f1': 0.695, 'eval_precision': 0.7433155080213903, 'eval_recall': 0.6525821596244131, 'eval_runtime': 3.4309, 'eval_samples_per_second': 40.515, 'eval_steps_per_second': 5.246, 'epoch': 3.0}
{'train_runtime': 1771.2377, 'train_samples_per_second': 4.453, 'train_steps_per_second': 0.557, 'train_loss': 0.4209353554212576, 'epoch': 3.0}


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

In [11]:
predictions = trainer.predict(TESTING_SET)
probs = sigmoid(torch.from_numpy(predictions.predictions))
# print("PROBS: ", probs)
# print("LABELS: ", torch.tensor(tokenized_test['label'])) # trues
print(f"RESULTS LOG:")
nums = np.round(np.linspace(.3, .7, 40), 2)
final_nums = np.append(nums, 0.5)

best_thresh = 0
best_f1 = 0
for curr_thresh in final_nums:
# binarize predictions
   binary_predictions = (probs >= curr_thresh).long()
   
   # print("PREDS: ", binary_predictions)

   curr_f1 = f1_score(y_true=TESTING_SET['label'], y_pred=binary_predictions, average='weighted')
   if curr_f1 > best_f1:
      best_f1 = curr_f1
      best_thresh = curr_thresh
# print("THRESH = ", curr_thresh, " F1 = ", curr_f1)
print(f"F1: {best_f1}")
print(best_thresh)

for thresh in [0.35, 0.4, 0.45, 0.5]:
# binarize predictions
   binary_predictions = (probs >= thresh).long()
   print("THRESH = ", thresh)
   # print("PREDS: ", binary_predictions)

   print(f1_score(y_true=TESTING_SET['label'], y_pred=binary_predictions, average='weighted'))
   print(f1_score(y_true=TESTING_SET['label'], y_pred=binary_predictions, average='micro'))
   print(f1_score(y_true=TESTING_SET['label'], y_pred=binary_predictions, average='macro'))
   print("==============")

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['label']

In [ ]:
# import pandas as pd


# predictions = trainer.predict(tokenized_dev) # logits
# probs = torch.sigmoid(torch.from_numpy(predictions.predictions)) # percentage probabilities
# print("PROBS: ", probs)
# binary_predictions = (probs >= 0.35).long()
# print(binary_predictions)

# # Convert tensors to integers
# binary_predictions_list = binary_predictions.tolist()


# ids = []
# for i in range(0, len(binary_predictions_list)):
#   ids.append("text" + str(i))

# data = {
#     "id": ids,
#     "Anger": [pred[0] for pred in binary_predictions_list],
#     "Fear": [pred[1] for pred in binary_predictions_list],
#     "Joy": [pred[2] for pred in binary_predictions_list],
#     "Sadness": [pred[3] for pred in binary_predictions_list],
#     "Surprise": [pred[4] for pred in binary_predictions_list],
# }

# df = pd.DataFrame(data)
# df.to_csv("pred_eng_a.csv", index=False) # drop the index column

  0%|          | 0/15 [00:00<?, ?it/s]

PROBS:  tensor([[0.3542, 0.8581, 0.0265, 0.3875, 0.2405],
        [0.0391, 0.9395, 0.0265, 0.2670, 0.6871],
        [0.0251, 0.5663, 0.1758, 0.0363, 0.5725],
        [0.0177, 0.3833, 0.2688, 0.3096, 0.0226],
        [0.0162, 0.2484, 0.3311, 0.0590, 0.2025],
        [0.0339, 0.7683, 0.0616, 0.4642, 0.0199],
        [0.8415, 0.5247, 0.0939, 0.6800, 0.2348],
        [0.0435, 0.2545, 0.3271, 0.2512, 0.0394],
        [0.0163, 0.1667, 0.5121, 0.0633, 0.0428],
        [0.1212, 0.9230, 0.0231, 0.7243, 0.0340],
        [0.0389, 0.1058, 0.7152, 0.0519, 0.3301],
        [0.0346, 0.9215, 0.0334, 0.7368, 0.0332],
        [0.5502, 0.6205, 0.0742, 0.3807, 0.4905],
        [0.2147, 0.8655, 0.0323, 0.1979, 0.6419],
        [0.0480, 0.0566, 0.9024, 0.1026, 0.1354],
        [0.0129, 0.7609, 0.0906, 0.2587, 0.1066],
        [0.5753, 0.4848, 0.1038, 0.6517, 0.1471],
        [0.0122, 0.5390, 0.1755, 0.0418, 0.2172],
        [0.2505, 0.9259, 0.0249, 0.2464, 0.5295],
        [0.5102, 0.7715, 0.0425, 0.6519, 0